In [ ]:
import gzip,os,tarfile,sys
sys.path.append(os.pardir+'/src')
from settings import *
from boto3.session import Session
import datetime
import traceback
import logging
import pandas as pd
from pandas import DataFrame
import xml.etree.ElementTree as et

In [ ]:
start_time = datetime.datetime.now()

# コマンドライン引数からダウンロードを開始する日付と範囲を取得
start_date = '20150101'
span = '365'
start = datetime.datetime.strptime(start_date, '%Y%m%d')

# S3へ接続
session = Session(aws_access_key_id=AWS_ACCESS_KEY_ID,aws_secret_access_key=AWS_SECRET_ACCESS_KEY)
s3 = session.resource('s3')
bucket = s3.Bucket(BUCKET_NAME)

# tagと日付リストを設定
tag = "EID42168_"
dateList = makeDateList(start, span)

for date in dateList:
    time = datetime.datetime.now()
    date = date[9:]
    try:
        downloadFile(bucket, tag, date)
        print(date + ' was done ' + str(datetime.datetime.now()-time))
    except Exception as e:
        print('error! ' + date)
        print(logging.error(traceback.format_exc()))

print('it taked ' + str(datetime.datetime.now() - start_time))

In [ ]:
# xml.gzファイルを解凍、xmlの構造を解析しCSVファイルを作成する関数
def convertToCSV(file_name):
    start = datetime.datetime.now()

    # 引数として渡されたtar.gzファイルを解凍し、オープン
    f = gzip.open(os.path.join(DATA_DIR,'originalData',file_name), 'r')

    # CSVファイルのカラムに対応する配列を初期化
    ids = []
    headlines = []
    timeofarrivals = []
    bodys = []
    #langs = []

    # xmlを解析し、rootを取得
    tree = et.parse(f)
    elem = tree.getroot()
    # エラーとなった記事をカウントする変数を初期化
    fail_cnt = 0

    # 解析したxmlから、記事単位で要素を取得
    contents = elem.getiterator('ContentT')

    for content in contents:
        try:
            # 記事の言語情報を取得し、日本語か英語の記事であれば以降の処理を実施
            lang = content.find(".//LanguageString").text
            #if lang == 'JAPANESE' or lang == 'ENGLISH':
            if lang == 'JAPANESE':
                if content.find(".//Body").text is not ' ':
                    # 言語、ID、タイトル、タイムスタンプを配列に格納
                    #langs.append(lang)
                    ids.append(content.find(".//Id/SUID").text)
                    headlines.append(content.find(".//Headline").text)
                    timeofarrivals.append(content.find(".//TimeOfArrival").text)
                    bodys.append(content.find(".//Body").text)
        except:
            # 読み取りに失敗した場合はカウント
            fail_cnt += 1
            #traceback.print_exc()
    
    print('fail_cnt:', fail_cnt)
    
    # カラムに対応する配列を用いてDataFrameを作成
    df = DataFrame({"Id":ids})
    df['Headline']=headlines
    df['Body']=bodys
    df['TimeOfArrival']=timeofarrivals
    df.drop_duplicates()
    # DataFrameをCSVファイルとして保存
    df.to_csv(os.path.join(DATA_DIR,'csvData1',file_name.replace(".xml.gz",".csv")),encoding='utf8',index=False)
    f.close()

    print('file_name:' + file_name + ' time:' + str(datetime.datetime.now()-start) + ' record_count:' + str(len(df)) + ' fail_count:' + str(fail_cnt))

In [ ]:
# dataフォルダ配下の全ファイルを取得
files = os.listdir(DATA_DIR+'/originalData')
for i in range(0,len(files)):
    convertToCSV(files[i])

In [ ]:
＃人名抽出方法の問題、ニックネームとかもね